In [1]:
import wandb
from wandb.keras import WandbMetricsLogger

import tensorflow as tf
from tensorflow import keras
from keras import layers
# from tensorflow.keras.layers 
import pickle 
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

2023-02-02 18:48:53.905320: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
(ds_cifar10_train, ds_cifar10_test), ds_cifar10_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    data_dir='/Users/herbertzhang/Desktop/BU/CS549/ml-549-course/homeworks/hw1datasets',
    shuffle_files=True, # load in random order
    as_supervised=True, # Include labels
    with_info=True, # Include info
)

    # Optionally uncomment the next 3 lines to visualize random samples from each dataset
    #fig_train = tfds.show_examples(ds_cifar10_train, ds_cifar10_info)
    #fig_test = tfds.show_examples(ds_cifar10_test, ds_cifar10_info)
    #plt.show()  # Display the plots

def normalize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    return tf.cast(image, tf.float32) / 255., label

# Prepare cifar10 training dataset
ds_cifar10_train = ds_cifar10_train.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_cifar10_train = ds_cifar10_train.cache()     # Cache data
ds_cifar10_train = ds_cifar10_train.shuffle(ds_cifar10_info.splits['train'].num_examples)
ds_cifar10_train = ds_cifar10_train.batch(32)  # <<<<< To change batch size, you have to change it here
ds_cifar10_train = ds_cifar10_train.prefetch(tf.data.AUTOTUNE)

# Prepare cifar10 test dataset
ds_cifar10_test = ds_cifar10_test.map(normalize_img, num_parallel_calls=tf.data.AUTOTUNE)
ds_cifar10_test = ds_cifar10_test.batch(32)    # <<<<< To change batch size, you have to change it here
ds_cifar10_test = ds_cifar10_test.cache()
ds_cifar10_test = ds_cifar10_test.prefetch(tf.data.AUTOTUNE)



In [5]:

# Define the model here
model = tf.keras.models.Sequential([
    keras.Input(shape=(32, 32, 3)),
    #####################################
    layers.Conv2D(64, (3,3)),
    layers.Activation("relu"), 
    layers.MaxPooling2D(pool_size=(2,2)),
    
    layers.Conv2D(64, (3,3)),
    layers.Activation("relu"), 
    layers.MaxPooling2D(pool_size=(2,2)),
    
    # vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    tf.keras.layers.Dense(10)
])

In [6]:

# Log the training hyper-parameters for WandB
# If you change these in model.compile() or model.fit(), be sure to update them here.
wandb.config = {
    #####################################
    # Edit these as desired
    # vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
    "learning_rate": 0.001,
    "optimizer": "adam",
    "epochs": 5,
    "batch_size": 32
    # ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
}

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=.001),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)

history = model.fit(
    ds_cifar10_train,
    epochs=10,
    validation_data=ds_cifar10_test,
    # callbacks=[WandbMetricsLogger()]
)
print(model.summary())

Error: You must call `wandb.init()` before WandbMetricsLogger()